In [2]:
import os
import torch
from torch import nn
from torch.nn import functional as F

import numpy as np
device ="cpu"
print(f"Using {device} device")

Using cpu device


In [4]:
import datetime
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')

import datetime

from finrl import config
from finrl import config_tickers
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader

from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts

from StockPortfolioEnv import StockPortfolioEnv as env

import sys
sys.path.append("../FinRL-Library")

import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

df = YahooDownloader(start_date = '2008-01-01',
                     end_date = '2022-06-01',
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

fe = FeatureEngineer(
    use_technical_indicator=True,
    use_turbulence=False,
    user_defined_feature=False
)

df = fe.preprocess_data(df)
# add covariance matrix as states
df=df.sort_values(['date','tic'],ignore_index=True)
df.index = df.date.factorize()[0]
cov_list = []
return_list = []

# look back is one year
lookback=252
for i in range(lookback,len(df.index.unique())):
  data_lookback = df.loc[i-lookback:i,:]
  price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
  return_lookback = price_lookback.pct_change().dropna()
  return_list.append(return_lookback)

  covs = return_lookback.cov().values 
  cov_list.append(covs)
df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list,'return_list':return_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)
train = data_split(df, '2009-04-01','2016-04-01')


stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")
tech_indicator_list = ['macd', 'rsi_30', 'cci_30', 'dx_30']
feature_dimension = len(tech_indicator_list)
print(f"Feature Dimension: {feature_dimension}")
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "transaction_cost_pct": 0, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": tech_indicator_list, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-1
    
}




[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [5]:
train = data_split(df, '2009-04-01','2016-06-30')
validation = data_split(df,'2016-06-30','2019-06-30')
test_set = data_split(df,'2019-07-01','2022-05-31')

e_train_gym = env(df = train, **env_kwargs)
e_validation_gym = env(df = validation,**env_kwargs)
e_test_gym = env(df=test_set,**env_kwargs)
env_train, test_obs = e_train_gym.get_sb_env()

In [6]:
import numpy as np
import gym
from collections import deque
import random


class OUNoise(object):
    def __init__(self, action_space, mu=0.0, theta=0.15, max_sigma=0.3, min_sigma=0.3, decay_period=100000):
        self.mu           = mu
        self.theta        = theta
        self.sigma        = max_sigma
        self.max_sigma    = max_sigma
        self.min_sigma    = min_sigma
        self.decay_period = decay_period
        self.action_dim   = action_space.shape[0]
        self.low          = action_space.low
        self.high         = action_space.high
        self.reset()
        
    def reset(self):
        self.state = np.ones(self.action_dim) * self.mu
        
    def evolve_state(self):
        x  = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.random.randn(self.action_dim)
        self.state = x + dx
        return self.state
    
    def get_action(self, action, t=0):
        #print('Low',self.low)
        #print('High',self.high)
        ou_state = self.evolve_state()
        self.sigma = self.max_sigma - (self.max_sigma - self.min_sigma)# * min(1.0, t / self.decay_period)
        #print('Ou',action+ou_state)
        #print('action clip',np.clip(action + ou_state, self.low, self.high))
        return np.clip(action + ou_state, -1, 1)
        #return np.clip(action,self.low,self.high)



class NormalizedEnv(gym.ActionWrapper):
    """ Wrap action """

    def _action(self, action):
        act_k = (self.action_space.high - self.action_space.low)/ 2.
        act_b = (self.action_space.high + self.action_space.low)/ 2.
        return act_k * action + act_b

    def _reverse_action(self, action):
        act_k_inv = 2./(self.action_space.high - self.action_space.low)
        act_b = (self.action_space.high + self.action_space.low)/ 2.
        return act_k_inv * (action - act_b)
        

class Memory:
    def __init__(self, max_size):
        self.max_size = max_size
        self.buffer = deque(maxlen=max_size)
    
    def push(self, state, action, reward, next_state, done):
        experience = (state, action, np.array([reward]), next_state, done)
        self.buffer.append(experience)

    def sample(self, batch_size):
        state_batch = []
        action_batch = []
        reward_batch = []
        next_state_batch = []
        done_batch = []

        batch = random.sample(self.buffer, batch_size)

        for experience in batch:
            state, action, reward, next_state, done = experience
            state_batch.append(state)
            action_batch.append(action)
            reward_batch.append(reward)
            next_state_batch.append(next_state)
            done_batch.append(done)
        
        return state_batch, action_batch, reward_batch, next_state_batch, done_batch

    def __len__(self):
        return len(self.buffer)

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.autograd
from torch.autograd import Variable

class Critic(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Critic, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, 150)
        self.linear3 = nn.Linear(150, 28)
        self.linear4 = nn.Linear(28,1)

    def forward(self, state, action):
        """
        Params state and actions are torch tensors
        """
        x = torch.cat([state, action], 1)
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = self.linear4(x)

        return x

class Actor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, learning_rate = 1e-4):
        super(Actor, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, 150)
        self.linear3 = nn.Linear(150, output_size)
        
    def forward(self, state):
        """
        Param state is a torch tensor
        """
        x = F.relu(self.linear1(state))
        x = F.relu(self.linear2(x))
        x = F.tanh(self.linear3(x)).clone()

        return x

In [8]:
import torch
import torch.autograd
import torch.optim as optim
import torch.nn as nn


class DDPGagent:
    def __init__(self, state_dim,action_dim,buffer, hidden_size=400, actor_learning_rate=1e-4, critic_learning_rate=1e-4, gamma=0.99, tau=0.005):
        # Params
        self.num_states = state_dim
        self.num_actions = action_dim
        self.gamma = gamma
        self.tau = tau

        # Networks
        self.actor = Actor(self.num_states, hidden_size, self.num_actions)
        self.actor_target = Actor(self.num_states, hidden_size, self.num_actions)
        self.critic = Critic(self.num_states + self.num_actions, hidden_size, self.num_actions)
        self.critic_target = Critic(self.num_states + self.num_actions, hidden_size, self.num_actions)

        for target_param, param in zip(self.actor_target.parameters(), self.actor.parameters()):
            target_param.data.copy_(param.data)

        for target_param, param in zip(self.critic_target.parameters(), self.critic.parameters()):
            target_param.data.copy_(param.data)
        
        # Training
        self.memory = buffer       
        self.critic_criterion  = nn.MSELoss()
        self.actor_optimizer  = optim.Adam(self.actor.parameters(), lr=actor_learning_rate)
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=critic_learning_rate)
    
    def get_action(self, state):
        state = Variable(torch.from_numpy(state).float().unsqueeze(0))
        action = self.actor.forward(state)
        #print('Action_inside',action)
        action = action.detach().numpy()[0]
        return action
    
    def get_action_clip(self,state):
        state = Variable(torch.from_numpy(state).float().unsqueeze(0))
        action = self.actor.forward(state)
        #print('Action_inside',action)
        action = action.detach().numpy()[0]
        #action = np.clip(action,0,1)
        return action
    
    
    def update(self, batch_size):
        states, actions, rewards, next_states, _ = self.memory.sample(batch_size)
        #print('Next_states',next_states)
        
        states = torch.FloatTensor(states)
        actions = torch.FloatTensor(actions)
        rewards = torch.FloatTensor(rewards)
        next_states = torch.FloatTensor(next_states)
        #print('States',states.size())
    
        # Critic loss        
        Qvals = self.critic.forward(states, actions)
        next_actions = self.actor_target.forward(next_states).clamp(-1,1)
        #print('Next_Actions',next_actions)
        #print('Next_actions',next_actions)
        next_Q = self.critic_target.forward(next_states, next_actions.detach())
        #print('Rewards '+str(rewards)+' gamma '+str(self.gamma)+' next_ q' +str(next_Q))
        
        Qprime = rewards + self.gamma * next_Q
        
        #print('QVals',Qvals)
        #print('Qprime',Qprime)
        critic_loss = self.critic_criterion(Qvals, Qprime)
        #critic_loss = sum(F.mse_loss(current_q, Qprime) for current_q in Qvals)
        #print('Critic_loss',critic_loss)
        # Actor loss
        policy_loss = -self.critic.forward(states, self.actor.forward(states)).mean() 
        
        #print('Policy_loss',policy_loss)
        #print('critic Loss',critic_loss)
        # update networks
        self.actor_optimizer.zero_grad()
        policy_loss.backward()
        self.actor_optimizer.step()

        self.critic_optimizer.zero_grad()
        critic_loss.backward() 
        self.critic_optimizer.step()

        # update target networks 
        for target_param, param in zip(self.actor_target.parameters(), self.actor.parameters()):
            target_param.data.copy_(param.data * self.tau + target_param.data * (1.0 - self.tau))
       
        for target_param, param in zip(self.critic_target.parameters(), self.critic.parameters()):
            target_param.data.copy_(param.data * self.tau + target_param.data * (1.0 - self.tau))
    
    def save_model(self, path,index,timestep):
        torch.save(self.actor.state_dict(), path+'_actor_'+str(index)+'_'+str(timestep))
        torch.save(self.actor_target.state_dict(), path+'_target_actor_'+str(index)+'_'+str(timestep))
        torch.save(self.critic.state_dict(), path+'_critic_'+str(index)+'_'+str(timestep))
        torch.save(self.critic_target.state_dict(), path+'_target_critic_'+str(index)+'_'+str(timestep))
        
        

    def load_model(self, path,index,timestep):
        self.actor.load_state_dict(torch.load(path+'_actor_'+str(index)+'_'+str(timestep)))
        self.actor_target.load_state_dict(torch.load(path+'_target_actor_'+str(index)+'_'+str(timestep)))
        self.critic.load_state_dict(torch.load(path+'_critic_'+str(index)+'_'+str(timestep)))
        self.critic_target.load_state_dict(torch.load(path+'_target_critic_'+str(index)+'_'+str(timestep)))
        
        self.actor.eval()
        self.actor_target.eval()
        self.critic.eval()
        self.critic_target.eval()


In [9]:
def normalized_action(action):
    low  = 0
    high = 1
    action = low + (action + 1.0) * 0.5 * (high - low)
    action = np.clip(action, low, high)

    return action

In [11]:
import torch as th
import sys
import gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def train_ensemble(number_of_agents, seed,env_train):
    seed = seed
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    
    k = number_of_agents
    Buffer = Memory(max_size = 1000000)
    e_train = NormalizedEnv(e_train_gym)

    env_train,test_obs = e_train.get_sb_env()
    agents = list()
    for i in range(k):
        agents.append(DDPGagent(state_dim = 896,action_dim=28,buffer = Buffer ))




    
    noise = OUNoise(env_train.action_space)
    batch_size = 256
    rewards = []
    avg_rewards = []
    env_train.reset()
    noise.reset()
    path ='MultiDDPG/17Aug/'
    
    
    #from DDPG import DDPGagent
    #from utils import *

    #numer of agents 
    rewards = []

    for episode in range(100000):


        obs_tensor = th.tensor(test_obs,requires_grad=False)
        state =th.flatten(obs_tensor).cpu().detach().numpy()

        index_agent = episode % k




        action = agents[index_agent].get_action(state)
        #print('Action_1',action)


        action = noise.get_action(action)
        action_normalized = normalized_action(action)

        #print('Daniel')
        test_obs, reward, done, _ = env_train.step(np.array([action_normalized]))
        #print('Action',action)
        #print('Reward',reward)

        new_obs_tensor = th.tensor(test_obs,requires_grad=False)
        new_state =th.flatten(new_obs_tensor).cpu().detach().numpy()
        Buffer.push(state, action, reward, new_state, done)
        rewards.append(reward)
        if len(Buffer) > batch_size and episode % 1024 == 0 and episode>10000:
            print('Updating')
            for _ in range(100) : 

                for agent in agents : 
                    #print('Updating')
                    agent.update(batch_size)   

       




        if done[0]:
            print('Hier')
            env_train,test_obs = e_train.get_sb_env()
            env_train.reset()

        
    return agents,rewards









In [12]:
agents1,rewards1 = train_ensemble(4,150,e_train_gym)
agents2,rewards2 = train_ensemble(4,31202,e_train_gym)
agents3,rewards3 = train_ensemble(4,42,e_train_gym)
agents4,rewards4 = train_ensemble(4,893,e_train_gym)
agents5,rewards5 = train_ensemble(4,98927,e_train_gym)
agents6,rewards6 = train_ensemble(4,2912,e_train_gym)
agents7,rewards7 = train_ensemble(4,8723,e_train_gym)
agents8,rewards8 = train_ensemble(4,1233,e_train_gym)
agents9,rewards9 = train_ensemble(4,596,e_train_gym)
agents10,rewards10 = train_ensemble(4,1,e_train_gym)
agents11,rewards11 = train_ensemble(4,10,e_train_gym)
agents12,rewards12= train_ensemble(4,100,e_train_gym)
agents13,rewards13= train_ensemble(4,2,e_train_gym)
agents14,rewards14= train_ensemble(4,20,e_train_gym)
agents15,rewards15 = train_ensemble(4,200,e_train_gym)



/home/ml-djonatan/anaconda3/envs/empirical/lib/python3.7/site-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


begin_total_asset:1000000
end_total_asset:3409521.883116278
Sharpe:  1.1506013963228752
Hier


KeyboardInterrupt: 

In [ ]:
def test(agent,env,seed):
    seed = seed
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    test_env, test_obs = env.get_sb_env()
    

    account_memory = []
    actions_memory = []
    #         state_memory=[] #add memory pool to store states
    test_env.reset()
    for i in range(len(env.df.index.unique())):
    
        
        obs_tensor = th.tensor(test_obs,requires_grad=False)
        state =th.flatten(obs_tensor).cpu().detach().numpy()
        
    
    
        
        action = agent.get_action_clip(state)
        action = normalized_action(action)
        
        
        
       
        test_obs, reward, done, _ = test_env.step(np.array([action]))
        
        
    
        if i == (len(env.df.index.unique()) - 2):
            print('hier')

            account_memory = test_env.env_method(method_name="save_asset_memory")
            actions_memory = test_env.env_method(method_name="save_action_memory")
    #                 state_memory=test_env.env_method(method_name="save_state_memory") # add current state to state memory
        if done[0]:
            print("hit end!")
            break
    return account_memory,actions_memory